In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Set Contest

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12.csv");

In [ ]:
var scenario = "IRU";
var year = 2020;
var month = 12;
var reportingNode = "CH";
var dataNode = "DT1.1";
var format = ImportFormats.Cashflow;
Func<IfrsVariable, bool> filter = x => x.AmountType == "NIC" && x.DataNode == dataNode;

In [ ]:
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                   p.Year == year &&
                                                                                                   p.Month == month &&
                                                                                                   p.Scenario == scenario).ToArrayAsync()).Single().Id;
var defaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                    p.Year == year &&
                                                                                                    p.Month == month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;


# Import Best Estimate and Scenario Cashflows

In [ ]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);  
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);
var defaultVar = Workspace.Query<IfrsVariable>().ToArray();
defaultVar.GroupBy(x => x.Partition).Select(g => g.Key)

In [ ]:
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv");
var scenarioVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();
(scenarioVar.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().NotBe(0)

In [ ]:
var log = await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario_BE.csv");
var mirrorBE = Workspace.Query<IfrsVariable>().Where(filter);
(mirrorBE.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().Be(0)

# Import Best Estimate and Scenario Actuals

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12.csv");
var defaultVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12_Scenario.csv");
var scenarioVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();
(scenarioVar.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().NotBe(0)

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12_Scenario_BE.csv");
var mirrorBE = Workspace.Query<IfrsVariable>().Where(filter);
(mirrorBE.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().Be(0)